In [0]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.preprocessing.sequence import pad_sequences
import sys
import re

In [0]:
# try:
#     device_name = os.environ['COLAB_TPU_ADDR']
#     TPU_ADDRESS = 'grpc://' + device_name
#     print('Found TPU at: {}'.format(TPU_ADDRESS))
# except KeyError:
#     print('TPU not found')

In [4]:
from google.colab import files
uploadedFile = files.upload()


Saving alice.txt to alice (1).txt


In [0]:
# load ascii text and covert to lowercase
filename = "alice.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()
#remove punctuation
raw_text = re.sub(r'[^\w\s]','',raw_text)


In [0]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [7]:
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  136186
Total Vocab:  31


In [33]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
#adding padding sequence  
dataXpad = pad_sequences(dataX, maxlen=seq_length)  
n_patterns = len(dataXpad)
print("Total Patterns: ", n_patterns)

Total Patterns:  136086


In [0]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [10]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True,dropout=0.1))
model.add(Dropout(0.1))
model.add(LSTM(256))
model.add(Dense(y.shape[1], activation='softmax'))
model.summary()


W0721 17:53:33.475324 140635696752512 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0721 17:53:33.494093 140635696752512 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0721 17:53:33.496808 140635696752512 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0721 17:53:33.658560 140635696752512 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0721 17:53:33.668927 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100, 256)          264192    
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 256)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dense_1 (Dense)              (None, 31)                7967      
Total params: 797,471
Trainable params: 797,471
Non-trainable params: 0
_________________________________________________________________


In [11]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

model.compile(loss='categorical_crossentropy', optimizer='adam')

W0721 17:53:34.193724 140635696752512 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0721 17:53:34.220710 140635696752512 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



In [0]:
#def model_to_tpu(model):
#    return tf.contrib.tpu.keras_to_tpu_model( model, strategy=tf.contrib.tpu.TPUDistributionStrategy(tf.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS)))


In [0]:
#tpu_model = model_to_tpu(model)

In [13]:
model.fit(X, y, epochs=100, batch_size=128, callbacks=callbacks_list)

W0721 17:53:34.363873 140635696752512 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/100
136086/136086 [==============================] - 291s 2ms/step - loss: 2.7347

Epoch 00001: loss improved from inf to 2.73474, saving model to weights-improvement-01-2.7347.hdf5
Epoch 2/100
136086/136086 [==============================] - 282s 2ms/step - loss: 2.3614

Epoch 00002: loss improved from 2.73474 to 2.36143, saving model to weights-improvement-02-2.3614.hdf5
Epoch 3/100
136086/136086 [==============================] - 282s 2ms/step - loss: 2.1475

Epoch 00003: loss improved from 2.36143 to 2.14750, saving model to weights-improvement-03-2.1475.hdf5
Epoch 4/100
136086/136086 [==============================] - 282s 2ms/step - loss: 2.0021

Epoch 00004: loss improved from 2.14750 to 2.00214, saving model to weights-improvement-04-2.0021.hdf5
Epoch 5/100
136086/136086 [==============================] - 280s 2ms/step - loss: 1.8726

Epoch 00005: loss improved from 2.00214 to 1.87259, saving model to weights-improvement-05-1.8726.hdf5
Epoch 6/100
136086/136086 [=======

In [35]:

# load the network weights
filename = "weights-improvement-22-1.2975.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(500):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print("\nDone.")

Seed:
" could think of
nothing better to say than his first remark it was the best butter
you know

alice ha "
d noleny toeezed uound tound oore of thrpled anlce whought lt herply intertupted inl gorugh hir soease yhen i had tounded tounde of tuch a louder tfparked tuite a little bnd taid to hereratly ano the sepembered tooe off tipgictly ano toeed oo a little bnd toutently ano toeeze sfplied anlce whought ierself sound and serpee onrsed up the whought souw 
iere taid the halled herself anl toeeze sfparked tound to whe tteen onse asd anlce whoue tourd thedhe ano toeed tourd the whe qther sffes one hldut

Done.
